In [1]:
# Installs
!conda install -c conda-forge geopandas --yes #Geopandas library
!conda install -c conda-forge geopy --yes #Geolocation library
!conda install -c conda-forge geocoder --yes #Geolocation library
!conda install -c conda-forge folium=0.5.0 --yes #Mapping library
!conda install -c conda-forge owslib --yes #Library to read WFS data

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    _libgcc_mutex:   0.1-main                          
    click-plugins:   1.1.1-py_0             conda-forge
    cligj:           0.5.0-py_0             conda-forge
    fiona:           1.7.13-py35_0          conda-forge
    geopandas:       0.5.1-py_0             conda-forge
    libspatialindex: 1.8.5-h20b78c2_2                  
    munch:           2.3.2-py_0             conda-forge
    pyproj:          1.9.5.1-py35h508ed2a_5 conda-forge
    readline:        7.0-ha6073c6_4                    
    rtree:           0.8.3-py35h2f9c1c0_0              
    shapely:         1.6.4-py35h164cb2d_1   conda-forge

The following packages will be UPDATED:

    conda:           4.3.31-py35_0                      --> 4.5.11-py35_0        conda-forge
    pycosat:         0.6.1-py35_1                    

In [2]:
# Imports
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopandas as gpd # librabry for GeoPandas

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from requests import Request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins

from owslib.wfs import WebFeatureService #Library for reading WFS data

print('Libraries imported.')

Libraries imported.


### Download Dutch area geolocation and demographic data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

Geolocation data sets for the four digits postal code can amongst others be found at PDOK (Public Services On the Map). PDOK is a platform for accessing geo data sets from Dutch governments. This is current and reliable data for both the public and private sectors. PDOK makes digital geo-information available as data services and files. The PDOK services are based on open data and are therefore freely available to everyone.

PDOK publishes datasets with Dutch postal codes geolocation, together with Dutch demographic data from the Dutch Central Bureau of Statistics (CBS).

See https://www.pdok.nl/-/publicatie-cbs-gegevens-naar-postcode

In [12]:
# URL for WFS backend
url = "https://geodata.nationaalgeoregister.nl/cbspostcode4/wfs"

# Initialize
wfs = WebFeatureService(url=url)

# Service provider 
print('Dataset identification title:', wfs.identification.title, '\n')

# Get WFS version
print('Dataset WFS version:', wfs.version, '\n')

# Available methods
print('Available methods:',[operation.name for operation in wfs.operations], '\n')

# Available data layers
print('Available data layers:',list(wfs.contents), '\n')

# Print all metadata of all layers
print('Metadata:')
for layer, meta in wfs.items():
    print(meta.__dict__, '\n')

Dataset identification title: CBS Postcodes4 

Dataset WFS version: 1.0.0 

Available methods: ['GetCapabilities', 'DescribeFeatureType', 'GetFeature'] 

Available data layers: ['cbspostcode4:postcode42015', 'cbspostcode4:postcode42016', 'cbspostcode4:postcode42017'] 

Metadata:
{'metadataUrls': [], 'crsOptions': [urn:ogc:def:crs:EPSG::28992], 'id': 'cbspostcode4:postcode42015', 'boundingBoxWGS84': (3.313578145224139, 47.975220611697736, 3.313639050016488, 47.97524922794849), 'boundingBox': (2.938747595140137, 50.56846156575199, 7.578926097286145, 53.62701798435816, urn:ogc:def:crs:EPSG::28992), 'defaulttimeposition': None, 'abstract': 'Gegevens per numeriek deel van de postcode voor het jaar 2015', 'auth': <Authentication shared=False username=None password=None cert=None verify=True>, 'timepositions': None, 'styles': None, 'keywords': ['CBS, postcode, inwoner, man, vrouw, leeftijd, herkomst, allochtoon, inkomen, ww, bijstand, woning, huur, voorziening, nabijheid, dichtheid'], 'title'

In [13]:
# Get data from WFS

# Fetch the last available year, which is 2017. See available layers in output above
layer = 'cbspostcode4:postcode42017'

# Specify the parameters for fetching the data
params = dict(service='WFS', version=wfs.version, request='GetFeature',
      typeName=layer, outputFormat='json')

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

# Read data from URL
pc4geo = gpd.read_file(q)

#Explore first lines
pc4geo.head()

,id,postcode,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
0,postcode42017.1,1011,9645,4990,4655,800,1195,3195,2920,1540,6380,1.5,4175,1305,345,500,75,60,30,10,6110,3605,115,115,800,390,670,370,45,5915,900,2060,825,6907,1,"(POLYGON ((122246.232 487910.177, 122259.064 4..."
1,postcode42017.2,1012,8240,4480,3760,465,1195,4195,1730,650,5960,1.4,4245,1175,190,260,50,50,30,10,5695,4725,20,50,85,490,225,105,-99997,5380,1090,2150,480,8410,1,"(POLYGON ((121995.0292 488243.2021, 121999.664..."
2,postcode42017.3,1013,21080,10550,10525,2640,1860,7685,6115,2775,12580,1.7,7525,2430,1020,1510,280,60,20,20,12825,6930,75,115,1625,1475,445,2005,160,12670,920,1990,2720,6224,1,"(POLYGON ((120506.2192 489494.5513, 120495.812..."
3,postcode42017.4,1014,645,375,270,65,45,400,115,20,330,1.7,170,105,10,45,15,60,20,20,270,20,5,5,5,20,185,-99997,25,225,1000,2160,80,2645,1,"(POLYGON ((120390.1126 489860.7457, 120387.401..."
4,postcode42017.5,1015,14810,7545,7265,1410,1465,5235,4365,2340,9740,1.5,6340,1925,535,860,125,60,30,10,9865,7275,100,235,755,1075,340,80,-99997,9385,1050,1960,1395,10975,1,"(POLYGON ((120665.6423 488535.5, 120668.7853 4..."


This data has area bounderies geolocation data, but not the center long/lat per postal code. Found this dataset for that, which gives long/lat for all four digit postcodes.
https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [19]:
#4PP LatLong in CSV dataset for Netherlands areas in LatLong geolocation
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1,1000,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.336243,4.869444,Postbus
1,2,1001,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus
2,3,1002,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
3,4,1003,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
4,5,1005,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus


In [20]:
neighlatlong.shape

(4699, 10)

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [21]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


Drop all cities not being the cities we want to research: Amsterdam, Rotterdam, The Hague and Utrecht

In [23]:
# List of cities to select
cityselect = ['amsterdam','rotterdam','den haag','utrecht']
neighlatlong = neighlatlong[neighlatlong.woonplaats.str.contains('|'.join(cityselect), case=False)]
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [24]:
#Oops, we have a city called Nieuw-Amsterdam completely somewhere else in the Netherlands. Drop that.
neighlatlong.drop(neighlatlong[neighlatlong.woonplaats == 'Nieuw-Amsterdam'].index, inplace=True)
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [25]:
neighlatlong.shape

(252, 10)

### CBS income data per postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on standardised income based on postal code.
See https://www.cbs.nl/nl-nl/maatwerk/2017/15/besteedbaar-inkomen-per-postcodegebied-2004-2014

In [10]:
#Excel has multi row header, so header is set to none. Only columns for latest year (2014) are imported.
#Columns are: postcode for first four digits postal code, huishoudens for number of households, gem_inkomen is average income, std_inkomen is standardised income corrected for householdsize
inkomen = pd.read_excel('https://www.cbs.nl/-/media/_excel/2017/15/besteedbaar-inkomen-postcode-2004-2014.xlsx',
                        sheet_name = 'Tabel1', header = None, usecols = [0, 11, 23, 35], skiprows = 10, skipfooter = 5)
inkomen.columns = ['postcode', 'huishoudens', 'gem_inkomen', 'std_inkomen' ]
inkomen.head()

,postcode,huishoudens,gem_inkomen,std_inkomen
0,1011,6.0,32,26.3
1,1012,5.7,26.2,22.4
2,1013,12.7,30.3,24.1
3,1014,0.3,35,28.5
4,1015,9.5,33.3,27.2


In [11]:
inkomen.shape

(4053, 4)

### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [26]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

Data downloaded!


In [27]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode
0,1011AB,105,3630400,36304,363
1,1011AB,106,3630400,36304,363
2,1011AB,107,3630400,36304,363
3,1011AB,110,3630400,36304,363
4,1011AB,112,3630400,36304,363


In [28]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

,wijkcode,wijk
0,300,Wijk 00
1,500,Wijk 00
2,900,Wijk 00 West
3,901,Wijk 01 Oost
4,1000,Wijk 00 Stad


In [29]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

,buurtcode,buurt
0,30000,Appingedam-Centrum
1,30001,Appingedam-West
2,30002,Appingedam-Oost
3,30007,Verspreide huizen Damsterdiep en Eemskanaal
4,30008,Verspreide huizen ten zuiden van Eemskanaal


In [30]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode,wijk,buurt
0,1011AB,105,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,106,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,107,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,110,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,112,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland


In [31]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

,PC6,wijk,buurt
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland


In [32]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

,PC6,wijk,buurt,postcode
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011


In [33]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.
2,1013,Haarlemmerbuurt,Westerdokseiland
3,1014,Westelijk Havengebied,Alfa-driehoek
4,1015,Grachtengordel-West,Langestraat e.o.


In [34]:
#Cast column postcode to INT instead of OBJECT as geolocation dataset has INT
postcode.postcode = postcode.postcode.astype('int64')

In [35]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


In [37]:
#Now merge with geolocation and demographic data
postcode = pd.merge(postcode, pc4geo, on='postcode')
postcode.head()

,postcode,wijk,buurt,id_x,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort,id_y,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres,postcode42017.1,9645,4990,4655,800,1195,3195,2920,1540,6380,1.5,4175,1305,345,500,75,60,30,10,6110,3605,115,115,800,390,670,370,45,5915,900,2060,825,6907,1,"(POLYGON ((122246.232 487910.177, 122259.064 4..."
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres,postcode42017.2,8240,4480,3760,465,1195,4195,1730,650,5960,1.4,4245,1175,190,260,50,50,30,10,5695,4725,20,50,85,490,225,105,-99997,5380,1090,2150,480,8410,1,"(POLYGON ((121995.0292 488243.2021, 121999.664..."
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres,postcode42017.3,21080,10550,10525,2640,1860,7685,6115,2775,12580,1.7,7525,2430,1020,1510,280,60,20,20,12825,6930,75,115,1625,1475,445,2005,160,12670,920,1990,2720,6224,1,"(POLYGON ((120506.2192 489494.5513, 120495.812..."
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres,postcode42017.4,645,375,270,65,45,400,115,20,330,1.7,170,105,10,45,15,60,20,20,270,20,5,5,5,20,185,-99997,25,225,1000,2160,80,2645,1,"(POLYGON ((120390.1126 489860.7457, 120387.401..."
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres,postcode42017.5,14810,7545,7265,1410,1465,5235,4365,2340,9740,1.5,6340,1925,535,860,125,60,30,10,9865,7275,100,235,755,1075,340,80,-99997,9385,1050,1960,1395,10975,1,"(POLYGON ((120665.6423 488535.5, 120668.7853 4..."


In [41]:
#Drop unneeded columns
postcode.drop(['id_x', 'id_y', 'alternatieve_schrijfwijzen', 'netnummer', 'soort'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9645,4990,4655,800,1195,3195,2920,1540,6380,1.5,4175,1305,345,500,75,60,30,10,6110,3605,115,115,800,390,670,370,45,5915,900,2060,825,6907,1,"(POLYGON ((122246.232 487910.177, 122259.064 4..."
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8240,4480,3760,465,1195,4195,1730,650,5960,1.4,4245,1175,190,260,50,50,30,10,5695,4725,20,50,85,490,225,105,-99997,5380,1090,2150,480,8410,1,"(POLYGON ((121995.0292 488243.2021, 121999.664..."
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21080,10550,10525,2640,1860,7685,6115,2775,12580,1.7,7525,2430,1020,1510,280,60,20,20,12825,6930,75,115,1625,1475,445,2005,160,12670,920,1990,2720,6224,1,"(POLYGON ((120506.2192 489494.5513, 120495.812..."
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,645,375,270,65,45,400,115,20,330,1.7,170,105,10,45,15,60,20,20,270,20,5,5,5,20,185,-99997,25,225,1000,2160,80,2645,1,"(POLYGON ((120390.1126 489860.7457, 120387.401..."
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14810,7545,7265,1410,1465,5235,4365,2340,9740,1.5,6340,1925,535,860,125,60,30,10,9865,7275,100,235,755,1075,340,80,-99997,9385,1050,1960,1395,10975,1,"(POLYGON ((120665.6423 488535.5, 120668.7853 4..."


In [42]:
postcode.shape

(249, 42)

### Show neighbourhoods on map for the cities of Amsterdam, Rotterdam, The Hague and Utrecht

In [43]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
print("Utrecht", Utrecht_coords)
g = geocoder.arcgis('Amsterdam, Netherlands')
amsterdam_coords = g.latlng
print("Amsterdam", amsterdam_coords)
g = geocoder.arcgis('Rotterdam, Netherlands')
rotterdam_coords = g.latlng
print("Rotterdam", rotterdam_coords)
g = geocoder.arcgis('Den Haag, Netherlands')
denhaag_coords = g.latlng
print("The Hague", denhaag_coords)

Utrecht [52.08965000000006, 5.114350000000059]
Amsterdam [52.36993000000007, 4.907880000000034]
Rotterdam [51.92282000000006, 4.478480000000047]
The Hague [52.08409000000006, 4.317320000000052]


In [44]:
#Extract only woonplaats is Utrecht
utrecht = postcode[postcode.woonplaats.str.contains('utrecht',case=False)]
utrecht.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
207,3511,Wijk 06 Binnenstad,"Lange Elisabethstraat, Mariaplaats en omgeving",Utrecht,Utrecht,Utrecht,52.089633,5.117682,8825,4320,4505,490,1460,3910,1860,1110,6080,1.4,4170,1315,145,370,70,70,20,10,4775,2555,90,-99997,525,1190,290,115,-99997,3730,530,2400,575,5167,1,"(POLYGON ((136430.772 456492.9581, 136431.56 4..."
208,3512,Wijk 06 Binnenstad,Breedstraat en Plompetorengracht en omgeving,Utrecht,Utrecht,Utrecht,52.089348,5.124238,8540,4335,4205,540,1845,3865,1615,680,5910,1.4,4285,1090,110,350,70,70,20,10,4365,3015,105,15,420,405,220,180,-99997,3240,550,2390,645,5725,1,"(POLYGON ((137189.451 456641.446, 137187.561 4..."
209,3513,Wijk 06 Binnenstad,Hoog-Catharijne NS en Jaarbeurs,Utrecht,Utrecht,Utrecht,52.097969,5.109309,5835,2885,2950,565,1130,2740,920,480,3675,1.6,2360,720,155,380,75,70,10,20,2800,1195,595,15,330,220,200,250,-99997,1840,930,2130,600,5145,1,"(POLYGON ((136097.04 457191.832, 136097.0951 4..."
210,3514,Wijk 04 Noordoost,Lauwerecht,Utrecht,Utrecht,Utrecht,52.100037,5.122910,7765,3685,4080,1140,1440,3060,1615,505,4440,1.7,2610,870,195,715,105,80,10,10,3330,2705,35,-99997,180,145,80,180,-99997,1995,1350,2500,490,5959,1,"(POLYGON ((137373.2533 457287.4308, 137379.473..."
211,3515,Wijk 04 Noordoost,Lauwerecht,Utrecht,Utrecht,Utrecht,52.102959,5.116839,5745,2740,3005,940,930,2250,1220,410,3085,1.8,1640,670,200,545,85,70,10,20,2575,1510,545,15,125,80,10,290,-99997,1655,940,2250,515,5535,1,"(POLYGON ((136393.578 457596.94, 136393.825 45..."


In [45]:
#Extract only woonplaats is Amsterdam
amsterdam = postcode[postcode.woonplaats.str.contains('amsterdam',case=False)]
amsterdam.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9645,4990,4655,800,1195,3195,2920,1540,6380,1.5,4175,1305,345,500,75,60,30,10,6110,3605,115,115,800,390,670,370,45,5915,900,2060,825,6907,1,"(POLYGON ((122246.232 487910.177, 122259.064 4..."
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8240,4480,3760,465,1195,4195,1730,650,5960,1.4,4245,1175,190,260,50,50,30,10,5695,4725,20,50,85,490,225,105,-99997,5380,1090,2150,480,8410,1,"(POLYGON ((121995.0292 488243.2021, 121999.664..."
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21080,10550,10525,2640,1860,7685,6115,2775,12580,1.7,7525,2430,1020,1510,280,60,20,20,12825,6930,75,115,1625,1475,445,2005,160,12670,920,1990,2720,6224,1,"(POLYGON ((120506.2192 489494.5513, 120495.812..."
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,645,375,270,65,45,400,115,20,330,1.7,170,105,10,45,15,60,20,20,270,20,5,5,5,20,185,-99997,25,225,1000,2160,80,2645,1,"(POLYGON ((120390.1126 489860.7457, 120387.401..."
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14810,7545,7265,1410,1465,5235,4365,2340,9740,1.5,6340,1925,535,860,125,60,30,10,9865,7275,100,235,755,1075,340,80,-99997,9385,1050,1960,1395,10975,1,"(POLYGON ((120665.6423 488535.5, 120668.7853 4..."


In [46]:
#Extract only woonplaats is The Hague
denhaag = postcode[postcode.woonplaats.str.contains('den haag',case=False)]
denhaag.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
81,2491,Wijk 44 Leidschenveen,De Velden,Den Haag,Den Haag,Zuid-Holland,52.069918,4.387614,280,155,125,40,45,30,120,45,110,2.6,30,25,-99997,50,-99997,80,10,20,80,15,-99997,-99997,-99997,5,35,20,-99997,-99997,1830,4710,10,1226,3,"(POLYGON ((87351.3955 454153.1568, 87335.45269..."
82,2492,Wijk 44 Leidschenveen,De Velden,Den Haag,Den Haag,Zuid-Holland,52.063672,4.412687,12645,6315,6330,2675,1625,2785,4200,1360,4530,2.7,835,1205,435,2045,105,60,10,30,4600,5,-99997,-99997,-99997,-99997,3760,825,-99997,1235,1040,3550,825,1429,3,"(POLYGON ((87798.963 454665.75, 87866.6023 454..."
83,2493,Wijk 44 Leidschenveen,De Lanen,Den Haag,Den Haag,Zuid-Holland,52.057377,4.403541,7915,4015,3900,2405,795,2680,1730,305,2615,3.0,415,450,240,1500,125,60,10,30,2550,25,15,5,-99997,-99997,350,2150,5,490,1050,3530,465,1399,3,"(POLYGON ((87024.39200000001 453685.246, 87086..."
84,2495,Wijk 41 Hoornwijk,Vlietzoom-West,Den Haag,Den Haag,Zuid-Holland,52.056302,4.361045,175,95,75,30,10,35,60,40,85,2.0,40,20,5,20,-99997,90,10,10,65,50,-99997,-99997,-99997,5,-99997,-99997,-99997,-99997,2250,3550,20,1312,3,"(POLYGON ((84225.951 451515.016, 84268.1910000..."
85,2496,Wijk 41 Hoornwijk,De Reef,Den Haag,Den Haag,Zuid-Holland,52.049285,4.371331,10910,5465,5445,2625,1370,3070,3090,750,4135,2.6,975,925,495,1705,135,60,10,30,4130,-99997,-99997,-99997,-99997,-99997,2980,1135,10,1500,0,3670,760,1739,2,"(POLYGON ((85720 452950, 85731 452943, 85732 4..."


In [47]:
#Extract only woonplaats is Rotterdam
rotterdam = postcode[postcode.woonplaats.str.contains('rotterdam',case=False)]
rotterdam.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
142,3011,Rotterdam Centrum,Stadsdriehoek,Rotterdam,Rotterdam,Zuid-Holland,51.917658,4.486852,15615,8180,7435,1240,2225,7375,3020,1755,9825,1.5,6075,2460,395,755,240,50,20,30,9165,40,2230,90,2525,1400,1450,1350,85,9135,90,2080,1235,6284,1,"(POLYGON ((93023.30009999999 437741.8521, 9303..."
143,3012,Rotterdam Centrum,Stadsdriehoek,Rotterdam,Rotterdam,Zuid-Holland,51.919405,4.475769,5660,2920,2745,435,895,2595,1020,715,3475,1.5,2185,755,185,290,105,40,20,40,3340,865,1025,100,480,360,-99997,495,15,3290,150,1920,595,6188,1,"(POLYGON ((92477.7307 437571.3672, 92480.5197 ..."
144,3013,Rotterdam Centrum,Cs Kwartier,Rotterdam,Rotterdam,Zuid-Holland,51.924289,4.469251,955,540,415,70,155,440,195,100,610,1.6,360,165,20,55,10,40,20,40,560,-99997,-99997,-99997,365,195,-99997,-99997,-99997,560,0,1970,55,6952,1,"(POLYGON ((91985.617 437556.413, 91984.7684 43..."
145,3014,Rotterdam Centrum,Cool,Rotterdam,Rotterdam,Zuid-Holland,51.919772,4.466089,7805,3945,3855,1120,1285,2395,1910,1095,4055,1.9,2205,645,550,590,65,20,10,70,3720,1650,-99997,10,785,1190,40,25,15,3620,940,1990,1595,6404,1,"(POLYGON ((91931.644 437513.552, 91933.754 437..."
146,3015,Rotterdam Centrum,Oude Westen,Rotterdam,Rotterdam,Zuid-Holland,51.911750,4.468044,2325,1190,1130,205,420,905,510,285,1480,1.6,920,330,55,145,25,60,10,30,1275,745,-99997,-99997,245,150,125,-99997,-99997,1250,760,2240,185,5620,1,"(POLYGON ((92140.401 436802.911, 92140.599 436..."


In [53]:
#Create map of Utrecht with neighbourhoods marked
map_utrecht = folium.Map(location=[Utrecht_coords[0], Utrecht_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(utrecht['latitude'], utrecht['longitude'], utrecht['wijk'], utrecht['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_utrecht)  
    
map_utrecht

In [49]:
#Create map of Amsterdam with neighbourhoods marked
map_amsterdam = folium.Map(location=[amsterdam_coords[0], amsterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(amsterdam['latitude'], amsterdam['longitude'], amsterdam['wijk'], amsterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

In [51]:
#Create map of Den Haag with neighbourhoods marked
map_denhaag = folium.Map(location=[denhaag_coords[0], denhaag_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(denhaag['latitude'], denhaag['longitude'], denhaag['wijk'], denhaag['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denhaag)  
    
map_denhaag

In [52]:
#Create map of Rotterdam with neighbourhoods marked
map_rotterdam = folium.Map(location=[rotterdam_coords[0], rotterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(rotterdam['latitude'], rotterdam['longitude'], rotterdam['wijk'], rotterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rotterdam)  
    
map_rotterdam